# 大语言模型预训练之全量参数微调

在当今的自然语言处理领域，全量参数微调（Fine-tuning）已成为释放大型预训练语言模型潜力的关键技术手段。然而，随着模型规模的扩大，这一过程对计算资源的要求也急剧上升。

在本次实践中，我们选择了一个拥有约8亿参数的模型进行全量参数微调。这一规模级别的模型，通常需要大约18GB的显存资源。尽管这在现代硬件上是可行的，但当模型规模扩大到80亿参数时，所需的显存将飙升至180GB，这种级别的资源消耗通常只有资金雄厚的企业才能承担。

进一步地，如果模型的规模达到惊人的800亿参数，那么所需的显存将达到庞大的1800GB。如此巨大的资源需求，即便是对于许多公司而言，也是一项极具挑战的任务，往往超出了他们的能力范围。因此，尽管全量参数微调在技术上可行，但在实际的应用和研究中，由于资源的限制，这样的实践相对较少。

鉴于此，大多数实际应用场景倾向于采用部分微调（Partial Fine-tuning）或迁移学习，这样既可以利用预训练模型中的知识，又可以针对性地调整模型以适应特定任务，而无需动用如全量微调那般庞大的计算资源。这种方法更加经济高效，同时也能产生令人满意的结果。

## 步骤1：导入相关包

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## 步骤2：加载数据集

In [2]:
ds = load_dataset("llm-wizard/alpaca-gpt4-data-zh",split="train")
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 48818
})

In [3]:
ds[:3]

{'instruction': ['保持健康的三个提示。', '三原色是什么？', '描述原子的结构。'],
 'input': ['', '', ''],
 'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。',
  '原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子中心，核外的电子围绕着原子核运动。\n\n原子结构具有层次性。原子核中，质子带正电，中子不带电（中性）。原子核非常小且致密，占据了原子总质量的绝大部分。电子带负电，通常围绕核运动，形成若干层次，称为壳层或电子层。电子数量与质子数量相等，使原子呈电中性。\n\n电子在每个壳层中都呈规律分布，并且不同壳层所能容纳的电子数也不同。在最里面的壳层一般只能容纳2个电子，其次一层最多可容纳8个电子，再往外的壳层可容纳的电子数逐层递增。\n\n原子核主要受到两种相互作用力的影响：强力和电磁力。强力的作用范围非常小，主要限制在原子核内，具有极强的吸引作用，使核子（质子和中子）紧密结合在一起。电磁力的作用范围较大，主要通过核外的电子与原子核相互作用，发挥作用。\n\n这就是原子的基本结构。原子内部结构复杂多样，不同元素的原子核中质子、中子数量不同

## 步骤3：数据预处理

### 1）获取分词器

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-800m-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-800m-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### 2）定义数据处理函数

In [5]:
def process_func(example):
    MAX_LENGTH = 256  # 定义最大长度为256
    input_ids, attention_mask, labels = [], [], []  # 初始化输入ID、注意力掩码和标签列表
    # 对指令和输入进行编码，返回输入ID和注意力掩码
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    # 对输出进行编码，返回输出ID和注意力掩码
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # 将指令和回应的输入ID合并
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 将指令和回应的注意力掩码合并
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 标签列表的前半部分是指令的长度个-100（表示这些位置的标签是被忽略的），后半部分是回应的输入ID
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 如果输入ID的长度大于最大长度，将其截断为最大长度
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    # 返回一个包含输入ID列表、注意力掩码列表和标签列表的字典
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

### 3）对数据进行预处理

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48818
})

### 4）数据格式检查

In [7]:
#检查一下数据格式，知识部分是否符合我们的需求
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 三原色是什么？\n\nAssistant: 三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。</s>'

In [8]:
## 检查一下数据格式，目标值是否符合我们的需求
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。</s>'

## 步骤4：创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-800m-zh",device_map="auto")

## 步骤5：配置训练参数

In [10]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/cache/finetuning/bloom-800m-zh-fft",# 指定模型训练结果的输出目录。
    per_device_train_batch_size=8,  # 指定每个设备（如GPU）上的批次大小
    gradient_accumulation_steps=8,  # 指定梯度累积步数。在本例子中，每8个步骤进行一次梯度更新。
    logging_steps=10, #指定日志记录的频率。在本例子中，每10个步骤记录一次日志
    num_train_epochs=1, #指定训练的总轮数。在本例子中，训练将进行1轮, 实际使用是会是多轮
    # max_steps=50  # 为节省时间，设置最大步数为 50
)

## 步骤6：创建训练器
data_collator在训练机器学习模型时，有以下作用：

1）数据转换：data_collator负责将输入的特征数据转换成统一形状和格式的张量(tensor)，这是为了便于模型进行统一的处理。

2）批量处理：它能够将多个数据样本整合成一个小批次(batch)的数据，这有助于提高模型训练的效率。

3）填充(padding)：在文本处理中，不同大小的输入需要被填充或截断到相同的长度以形成统一的形状，这对于很多自然语言处理模型来说是必要的，而DataCollatorWithPadding就是执行这一操作的常用collator。

> 综上所述，data_collator是连接数据处理与模型训练之间的重要桥梁，确保了数据的有效整理和组织，以便模型可以高效地从中学习。

In [11]:
trainer = Trainer(
    model=model,#指定训练模型
    args=args, #指定训练参数
    train_dataset=tokenized_ds, #指定数据集
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True) #指定数据收集器。其中tokenizer是分词器，padding=True表示对输入进行填充以保持批次大小一致。
)

## 步骤7：模型训练

In [12]:
trainer.train()

[2024-09-03 11:18:03,196] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss
10,2.445300
20,2.295500
30,2.341100
40,2.246900
50,2.269800
60,2.229600
70,2.225100
80,2.224500
90,2.211700
100,2.208000


TrainOutput(global_step=762, training_loss=2.068820762196238, metrics={'train_runtime': 2066.667, 'train_samples_per_second': 23.622, 'train_steps_per_second': 0.369, 'total_flos': 4.940441971133645e+16, 'train_loss': 2.068820762196238, 'epoch': 0.9988530231033917})

```bash
Tue Sep  3 11:19:32 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:31:00.0 Off |                  Off |
| 43%   68C    P2             409W / 450W |  19775MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
+---------------------------------------------------------------------------------------+
```

## 步骤8：模型推理

In [13]:
# 保存模型及训练状态
trainer.save_model("/root/autodl-tmp/cache/finetuning/bloom-800m-zh-fft")

In [22]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
result=pipe(ipt, max_length=256, do_sample=True, )
print(result[0]['generated_text'])

Human: 如何写好一个简历？

Assistant: 写好一个简历是一个复杂的过程，这里有一些关键步骤和技巧：

1. 把握信息：首先，你需要知道你的求职职位和应聘岗位。这一步非常重要，因为它将帮助确保你针对这个职位的应聘信息。

2. 收集信息：收集有关职位的信息，包括职位的要求、职责、技能要求、工作经验和其他需要的信息。

3. 编写简历：根据提供的信息和背景来编写简历。确保内容简洁明确，语言流畅，使用得体。

4. 设计图表和图片：使用图表和图片来描述你的技能或者经历，并且简洁明了地展示。可以尝试使用图表来更清楚地表达信息和图像。

5. 校对简历：仔细校对简历，确保所有内容都是正确的。可以联系招聘人员或招聘网站来检查应聘者的简历。

6. 整理简历：在招聘网站上提交简历后，不要忘记整理简历。将重要内容如职位、经验、联系信息等整理好，并归档。

7. 校对和修改：当简历提交后，招聘人员可能会通知你进行校对和修改。你可以与招聘人员合作，
